---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [5]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [125]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
#     states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
#               'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 
#               'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 
#               'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 
#               'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 
#               'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
#               'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
#               'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 
#               'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 
#               'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 
#               'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 
#               'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 
#               'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    
    #inv_map_states = {v: k for k, v in states.items()}
    
    filepath = 'university_towns.txt'
    state_var = ""
    region_var = ""
    towns = pd.DataFrame(columns=['State', 'RegionName'])
    
    with open(filepath) as fp:
        for cnt, line in enumerate(fp):
            if '[edit]' in line:
                state_pos = line.find('[')
                state_var = line[0:state_pos]
                state_var = state_var.rstrip()
            else:
                reg_pos = line.find('(')
                region_var = line[0:reg_pos]
                region_var = region_var.rstrip()
                towns = towns.append({'State': state_var, 'RegionName': region_var}, ignore_index=True)
                
    #towns = towns.replace({'State': inv_map_states})
    
    return towns

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [145]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    '''A quarter is a specific three month period, Q1 is January through March, Q2 is April through June, 
    Q3 is July through September, Q4 is October through December.
    
    A recession is defined as starting with two consecutive quarters of GDP decline, 
    and ending with two consecutive quarters of GDP growth.
    
    A recession bottom is the quarter within a recession which had the lowest GDP.'''
    
    recession_start = ""
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[219:286]
    gdp = gdp.drop(['Current-Dollar and "Real" Gross Domestic Product', 'Unnamed: 1', 'Unnamed: 2',
                   'Unnamed: 3'], axis = 1)
    #gdp.dropna(axis='columns')
    gdp.rename(columns = {'Unnamed: 4':'Quarter', 'Unnamed: 5':'GDP in billions of current dollars',
                         'Unnamed: 6':'GDP in billions of chained 2009 dollars'}, inplace=True)
    
    gdp.set_index('Quarter', inplace=True)
    
    lst_start = []
    lst_end = []
    recession = False
    
    for i in range(1, len(gdp)-1):
        if not recession and (gdp.iloc[i-1, 0] > gdp.iloc[i, 0] > gdp.iloc[i+1, 0]):
            recession = True
            lst_start.append(gdp.index[i-1])
        elif recession and (gdp.iloc[i-1, 0] < gdp.iloc[i, 0] < gdp.iloc[i+1, 0]):
            recession = False
            lst_end.append(gdp.index[i+1]) 
            
    #print(list(zip(lst_start, lst_end)))
    
    lst_start[0] = str(lst_start[0])
        
    return str(lst_start[0])

get_recession_start()

'2008q3'

In [146]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    '''A quarter is a specific three month period, Q1 is January through March, Q2 is April through June, 
    Q3 is July through September, Q4 is October through December.
    
    A recession is defined as starting with two consecutive quarters of GDP decline, 
    and ending with two consecutive quarters of GDP growth.
    
    A recession bottom is the quarter within a recession which had the lowest GDP.'''
    
    pd.set_option('display.max_rows', 30)
    
    recession_end = ""
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[219:286]
    gdp = gdp.drop(['Current-Dollar and "Real" Gross Domestic Product', 'Unnamed: 1', 'Unnamed: 2',
                   'Unnamed: 3'], axis = 1)
    #gdp.dropna(axis='columns')
    gdp.rename(columns = {'Unnamed: 4':'Quarter', 'Unnamed: 5':'GDP in billions of current dollars',
                         'Unnamed: 6':'GDP in billions of chained 2009 dollars'}, inplace=True)
    
    gdp.set_index('Quarter', inplace=True)
    
    lst_start = []
    lst_end = []
    recession = False
    
    for i in range(1, len(gdp)-1):
        if not recession and (gdp.iloc[i-1, 0] > gdp.iloc[i, 0] > gdp.iloc[i+1, 0]):
            recession = True
            lst_start.append(gdp.index[i-1])
        elif recession and (gdp.iloc[i-1, 0] < gdp.iloc[i, 0] < gdp.iloc[i+1, 0]):
            recession = False
            lst_end.append(gdp.index[i+1]) 
            
    #print(list(zip(lst_start, lst_end)))
    
    lst_end[0] = str(lst_end[0])
        
    return str(lst_end[0])

get_recession_end()

'2009q4'

In [147]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3
    
    A recession bottom is the quarter within a recession which had the lowest GDP.'''
    
    #pd.set_option('display.max_rows', None)
    
    recession_end = ""
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[219:286]
    gdp = gdp.drop(['Current-Dollar and "Real" Gross Domestic Product', 'Unnamed: 1', 'Unnamed: 2',
                   'Unnamed: 3'], axis = 1)
    #gdp.dropna(axis='columns')
    gdp.rename(columns = {'Unnamed: 4':'Quarter', 'Unnamed: 5':'GDP in billions of current dollars',
                         'Unnamed: 6':'GDP in billions of chained 2009 dollars'}, inplace=True)
    
    gdp.set_index('Quarter', inplace=True)
    
    lst_start = []
    lst_end = []
    recession = False
    
    for i in range(1, len(gdp)-1):
        if not recession and (gdp.iloc[i-1, 0] > gdp.iloc[i, 0] > gdp.iloc[i+1, 0]):
            recession = True
            lst_start.append(gdp.index[i])
        elif recession and (gdp.iloc[i-1, 0] < gdp.iloc[i, 0] < gdp.iloc[i+1, 0]):
            recession = False
            lst_end.append(gdp.index[i]) 
            
    #print(list(zip(lst_start, lst_end)))
    
    recession_df = gdp.loc[ '2008q4' : '2009q3' , : ]
    #print(recession_df)
    recession_bottom = recession_df.iloc[:,0].idxmin()
    
    return recession_bottom

get_recession_bottom()

'2009q2'

In [148]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
              'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 
              'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 
              'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 
              'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 
              'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
              'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
              'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 
              'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 
              'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 
              'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 
              'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 
              'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    
    housing_df = pd.read_csv('City_Zhvi_AllHomes.csv', index_col=0, skiprows=0)
    housing_df.drop(housing_df.iloc[:,5:50], inplace=True, axis=1)
    housing_df.replace({'State': states}, inplace=True)
    housing_df.set_index(['State', 'RegionName'], inplace=True)
    housing_df.sort_index(inplace=True)
    
    range_years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
    quarters = [('01','03'),('04','06'),('07','09'),('10','12')]
    quarters_2016 = [('01','03'),('04','06'),('07','08')]
    
    for year in range_years:
        if year != 2016:
            for quarter in range(len(quarters)):
                qstart = '{}-{}'.format(year, quarters[quarter][0])
                qend = '{}-{}'.format(year, quarters[quarter][1])
                qname = '{}q{}'.format(year, quarter+1)
            
                col = housing_df.loc[: , qstart:qend]
                housing_df[qname] = col.mean(axis=1, skipna=True)
                housing_df.drop(housing_df.iloc[:,3:6], inplace=True, axis=1)
        else:
            for quarter in range(len(quarters_2016)):
                qstart = '{}-{}'.format(year, quarters_2016[quarter][0])
                qend = '{}-{}'.format(year, quarters_2016[quarter][1])
                qname = '{}q{}'.format(year, quarter+1)
            
                col = housing_df.loc[: , qstart:qend]
                housing_df[qname] = col.mean(axis=1, skipna=True)
                if quarter != 2:
                    housing_df.drop(housing_df.iloc[:,3:5], inplace=True, axis=1) 
                else:
                    housing_df.drop(housing_df.iloc[:,3:7], inplace=True, axis=1)
    
    housing_df.drop(housing_df.iloc[:,:3], inplace=True, axis=1)
    
    return housing_df

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.666667   70266.666667   70300.000000   
          Brent             92933.333333   94333.333333   96166.666667   
          Brighton                   NaN            NaN            NaN   
          Brookwood         92566.666667   95100.000000   98866.666667   
...                                  ...            ...            ...   
Wisconsin Wilson           129033.333333  129366.666667  132433.333333   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        124366.666667  123666.666667  124133.333333   
          Wind Point       149233.333333  145266.666667  140866.666667   
          Wisconsin Dells  100000.000000  103400.000000  105000.000000   
          Wolf River        95166.666667   99333.333333  103800.000000   
          Woodruff                   NaN            NaN            NaN   
          Woodville                  NaN            NaN            NaN   
          Yorkville        139066.666667  134900.000000  135033.333333   
Wyoming   Bar Nunn          93033.333333   89666.666667   88900.000000   
          Burns            101533.333333  104566.666667  108366.666667   
          Casper            89233.333333   89600.000000   89733.333333   
          Cheyenne         116866.666667  120033.333333  121533.333333   
          Evansville       128033.333333  128766.666667  130833.333333   
          Pine Bluffs       93733.333333   95066.666667   94633.333333   

                                  2000q4         2001q1         2001q2  \
State     RegionName                                                     
Alabama   Adamsville        71966.666667   73466.666667   74000.000000   
          Alabaster        123700.000000  123233.333333  125133.333333   
          Albertville       74200.000000   75900.000000   76000.000000   
          Arab              82966.666667   84200.000000   84533.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81300.000000   85700.000000   87266.666667   
          Bayou La Batre    43666.666667   42500.000000   43333.333333   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        56066.666667   56833.333333   57600.000000   
          Boaz              71466.666667   72833.333333   71900.000000   
          Brent             98333.333333   96533.333333   98500.000000   
          Brighton                   NaN            NaN            NaN   
          Brookwood         99966.666667  101666.666667  103666.666667   
...                                  ...            ...            ...   
Wisconsin Wilson           137200.000000  143533.333333  143833.333333   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        126066.666667  132133.333333  139466.666667   
          Wind Point       147866.666

In [152]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    
    housing_df = convert_housing_data_to_quarters()
    recession_df = housing_df.loc[:, recession_start:recession_bottom]
    change = recession_df[recession_bottom]-recession_df[recession_start]
    recession_df['Change in Housing Prices'] = change
    recession_df.reset_index(inplace=True)
    
    university_town_df = get_list_of_university_towns()
    #university_town_df.set_index(['State', 'RegionName'], inplace=True)
    
    #uni_towns = pd.DataFrame(columns=['State', 'RegionName', '2008q4', '2009q1', '2009q2', 'Change in Housing Prices'])
    #non_uni_towns = pd.DataFrame(columns=['State', 'RegionName', '2008q4', '2009q1', '2009q2', 'Change in Housing Prices'])
    
    towns = pd.merge(university_town_df, recession_df, how='outer', indicator=True)
    towns.set_index(['State', 'RegionName'], inplace=True)

    uni_towns = towns[towns['_merge'] == 'both']
    uni_towns.drop('_merge', inplace=True, axis=1)
    nonuni_towns = towns[towns['_merge'] == 'right_only']
    nonuni_towns.drop('_merge', inplace=True, axis=1)
    
    uni_change_mean = uni_towns['Change in Housing Prices'].mean()
    nonuni_change_mean = nonuni_towns['Change in Housing Prices'].mean()
    
    uni_start_mean = uni_towns[recession_start].mean()
    nonuni_start_mean = nonuni_towns[recession_start].mean()
    
    uni_mean_ratio = uni_change_mean/uni_start_mean
    nonuni_mean_ratio = nonuni_change_mean/nonuni_start_mean
    
    from scipy import stats
    
    ttest,p = stats.ttest_ind(uni_towns.dropna()['Change in Housing Prices'], nonuni_towns.dropna()['Change in Housing Prices'])
    
    different = p < 0.01
        
    if uni_mean_ratio > nonuni_mean_ratio:
        better='university town'
    else:
        better='non-university town'
        
    answ = (different, p, better)
    
    return answ

run_ttest()

(-8766.7312661498727,
 -13090.096267054505,
 205559.10852713179,
 237285.22404598654,
 -0.042648225753483211,
 -0.055166082589776462)